In [ ]:
# Import the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Phase 1: ETL

# 1. Load the CSV file to use.

df_btc = pd.read_csv('csv files/coin_Bitcoin.csv') # Loading the CSV file

df_btc.head() # Shows the first 5 rows

In [ ]:
# 2. Convert the Date column to datetime and set is as the index.

df_btc['Date'] = pd.to_datetime(df_btc['Date']) # Formatted the data column to datetime
df_btc = df_btc.set_index('Date').sort_index() # Set the Date column as the index and sort it in chronological order.

In [ ]:
# 3. Key Transformations:

# 3.1 Calculate Daily Returns:

R_t = df_btc['Daily Return'] = df_btc['Close'].pct_change() * 100 # The df had the close value already, so the only needed thing was to change it to a percentage using .pct_change().

# 3.2 Calculate the Rolling Volatility (Standard Deviation) by 30 days:

CRV = df_btc['Volatility_30D'] = df_btc['Daily Return'].rolling(window=30).std()

# Now, it's necessary to remove the first row because it has the only NaN after creating the Daily Return column.

df_btc = df_btc.dropna()

In [ ]:
# Phase 2: Descriptive Analysis.

# 1. Price and Risk (Volatility) Line Plot.

plt.title('Bitcoin Closing Price History')

x = df_btc.index # Determine the X axis as the Date (Index)
plt.xlabel('Year')

y = df_btc['Close'] # Determine the Y axis as the closing value of each year.
plt.ylabel('Value of the Asset')

plt.plot(x, y)

In [ ]:
# 2. Risk timeline Line Plot.

x = df_btc.index # Determine the X axis and its label.
plt.xlabel('Year')

y = df_btc['Volatility_30D'] # Determine the Y axis and its label.
plt.ylabel('Volatility')

plt.title('Volatility Timeline') # Determine the title of the chart

plt.plot(x, y)

In [ ]:
# 3. Histogram Visualization KDE.

data = df_btc['Daily Return'].dropna()# This will be the data for the visualization.

plt.figure(figsize=(8, 6))

sns.histplot(data, bins=50, kde=True, color='deepskyblue', stat='density') # Bins are the amount of clusters of data, and the kde is for the density curve.

# Add the labels and titles.

plt.title('Distribution of Bitcoin Daily Returns', fontsize=10) # Main Title
plt.xlabel('Daily Return (%)') # X Label
plt.ylabel('Density') # Y Label
plt.show()

In [ ]:
# Phase 3: Trading Strategy Implementation

# 1. Calculate the Short-term Moving Average and the Long-term Moving Average.

df_btc['SMA'] = df_btc['Close'].rolling(window=50).mean() # In this case, the short-term moving average is being determined within 50 days.

df_btc['LMA'] = df_btc['Close'].rolling(window=200).mean() # On this one, we'll calculate the LMA based in 200 days.

# If we check the dataframe at this point it will show new np.NaN values that we have to drop.

df_btc = df_btc.dropna() # Now that the DataFrame has been cleaned, we can proceed with the next step.

In [ ]:
# 2. Generate the Signals (Buy-Sell).

df_btc['Signal'] = 0 # Generate a new column for the signals.

# If the SMA is above the LMA, the signal will become a "buy signal" represented by a 1:

df_btc.loc[df_btc['SMA'] > df_btc['LMA'], 'Signal'] = 1 # Representation of the buy signal.

# Now, its required to calculate the crossover to determine the sell signal (-1).

df_btc['Position'] = df_btc['Signal'].diff() # Created a new column to display the changes on the Signal column.

# This time a new NaN value appeared but this one won't affect the results of the Data Frame.

In [ ]:
# 3. Visualizations for Phase 3.

plt.figure(figsize=(8, 6))
ax = plt.gca()

# Plot 1: Closing Price.
ax.plot(df_btc.index, df_btc['Close'],
        color='gray', linewidth=2, label='Closing Price', alpha=0.6)

# Plot 2: Short-term Moving Average.
ax.plot(df_btc.index, df_btc['SMA'],
        color='deepskyblue', linewidth=1.8, label='SMA 50 Days')

# Plot 3: Long-term Moving Average.
ax.plot(df_btc.index, df_btc['LMA'],
        color='red', linewidth=1.8, label='LMA 200 Days')

# Buy signal markers (Green marker)
ax.scatter(df_btc.index[df_btc['Position'] == 1],
           df_btc['LMA'][df_btc['Position'] == 1],
           marker='^', s=150, color='green',
           label='Buy Signal', zorder=5)

# Sell signal marker. (Orange marker)
ax.scatter(df_btc.index[df_btc['Position'] == -1],
           df_btc['LMA'][df_btc['Position'] == -1],
           marker='v', s=150, color='orange',
           label='Sell Signal', zorder=5)

# Add the titles and labels for each plot.

ax.set_title('Bitcoin Trading Strategy', fontsize=10) # Main Title.
ax.set_xlabel('Date', fontsize=12) # Label for the X axis.
ax.set_ylabel('Price (USD)', fontsize=12) # Label for the Y axis.
ax.legend(loc='best', frameon=True, shadow=True, fancybox=True)
plt.show()